In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import heartpy as hp #이 모듈은 깔아야 import 할수 있음
import seaborn as sns
import scipy.stats as ss
import pywt

from sklearn.preprocessing import minmax_scale 

In [2]:
colname = ['ppg_grn', 'SKT', 'GSR', 'valence', 'arousal']
#data = pd.read_csv('WMinMaxs2e5.csv', header = None)
#data.columns = colname

MotherWavelet_ppg = pywt.Wavelet('db2')   # Mother wavelet (모함수) 지정
MotherWavelet_gsr = pywt.Wavelet('db3')

select  = 4                    # 특징추출 영역 고주파 영역부터 개수 지정 (d1~)

# 함수 정의
def rms(x):
    return np.sqrt(np.mean(x**2))

def crestF(x):
    return np.max(x) / rms(x)

def shapeF(x):
    return rms(x) / np.mean(x)

def impulseF(x):
    return np.max(x)/ np.mean(x)

def zero_out_detailed(coeff, level):
    for i in range(0, level+1):
        for j in range(0, len(coeff[i])):
            coeff[i][j] = 0

# Phasic = GSR - A10 
def get_phasic_amp_and_max(gsr_raw, recon_a10):    
    phasic = []
    domain_size = min(len(gsr_raw), len(recon_a10))
    for i in range(0, domain_size):
        phasic.append(gsr_raw[i] - recon_a10[i])
    # find amplitude and max 
    amp = max(phasic[0:domain_size]) - min(phasic[0:domain_size])
    max_val = max(phasic[0:domain_size])
    return amp, max_val

# SCVSR = A8 - A10
def get_scvsr_zcross(recon_a8, recon_a10):  
    A8minusA10 = [] 
    domain_size = min(len(recon_a8), len(recon_a10))
    for i in range(0, domain_size): 
        A8minusA10.append(recon_a8[i] - recon_a10[i])
    # normalize
    norm_scvsr = minmax_scale(A8minusA10)
    # consider offset 
    for i in range(0, len(norm_scvsr)):
        norm_scvsr[i] = norm_scvsr[i] - 0.5 
    # find zero cross   
    zero_cross_scvsr = np.nonzero(np.diff(norm_scvsr > 0))[0]
    rate_zero_cross_scvsr = zero_cross_scvsr.size
    return rate_zero_cross_scvsr

# SCSR = A6 - A10
def get_scsr_zcross(recon_a6, recon_a10): 
    A6minusA10 = []
    domain_size = min(len(recon_a6), len(recon_a10))
    for i in range(0, domain_size):
        A6minusA10.append(recon_a6[i] - recon_a10[i])
    # normalize 
    norm_scsr = minmax_scale(A6minusA10)
    # consider offset 
    for i in range(0, len(norm_scsr)):
        norm_scsr[i] = norm_scsr[i] - 0.5 
    #find zero cross 
    zero_cross_scsr = np.nonzero(np.diff(norm_scsr > 0))[0]
    rate_zero_cross_scsr = zero_cross_scsr.size
    return rate_zero_cross_scsr

#  data1 (길이:11940) 을 받아 
#  data1의 range가 stride 적용받으니 이 함수에서는 해당 range에 대한 NN, num50, p50, rmssd 만 반환하면 됨
def get_NN(rangedData, fs):
    working_data, measures = hp.process(rangedData['ppg_grn'].values, fs, calc_freq=False)
    RR_list = working_data['RR_list']
    RR_diff = working_data['RR_diff']
    NN = np.array(RR_list)
    return  NN # return NN 

In [5]:
# 결과값 딕셔너리로 반환 함수
def features2dictionary(fs, time_interval, data1, strd):

    #PPG 시간영역(0~5)
    meanNN = []
    medianNN = []
    SDNN = []
    pNN50 = []
    NN50 = []
    RMSSD = []
    
    #PPG 주파수영역(6~16)
    maxFPPG = []
    minFPPG = []
    meanFPPG = []
    stdFPPG = []
    maxminFPPG = []
    kurFPPG = []
    skewFPPG = []
    rmsFPPG = []
    crestFPPG = []
    shapeFPPG = []
    impulseFPPG = []
    

    #GSR 시간영역(17~27)
    maxGSR = []
    minGSR = []
    meanGSR = []
    stdGSR = []
    maxminGSR =[]
    kurGSR = []
    skewGSR = []
    rmsGSR = []
    crestGSR = []
    shapeGSR = []
    impulseGSR = []
    
    
    #GSR 주파수영역(28-31)
    zeroCrossScvsrFGSR = []
    zeroCrossScsrFGSR = []
    phasicMaxAmpFGSR = []
    phasicMaxValFGSR = []
    
    #SKT 시간영역(31~41)
    maxSKT = []
    minSKT = []
    meanSKT = []
    stdSKT = []
    maxminSKT = []
    kurSKT = []
    skewSKT = []
    rmsSKT = []
    crestSKT = []
    shapeSKT = []
    impulseSKT = []

    #valence arousal labeling(41~44)
    label_valence = []
    label_arousal = []
    valence = []
    arousal = []
    
    for i in range(0, int(time_interval/strd)):
        # Extract data in a range
        row_start = int(i*fs*strd)
        row_end = int(((i*fs*strd)+(time_interval)))
        data1 = data.iloc[row_start:row_end, 0:5]
        print('row start::',row_start, 'row end::', row_end)
        print('shape::',data1.shape)
        if(data1.shape == (0, 5)):
            break
        
        
        # PPG Time Domain Computation
        NN = get_NN(data1, fs)
        diff = []
        for j in range(0, len(NN) - 1):
            diff.append(abs(NN[j+1] - NN[j]))
        
        num50 = sum(np.array(diff) > 50)
        p50 = 0
        if len(diff) > 0:
            p50 = num50/len(diff)
        rmssd = np.sqrt(np.mean(sum(NN ** 2)))
        
        meanNN.append(np.mean(NN))
        medianNN.append(np.median(NN))
        SDNN.append(np.std(NN))
        NN50.append(num50)
        pNN50.append(p50)
        RMSSD.append(rmssd)

        LevelFour = 4
        PPGCoef = pywt.wavedec(data1['ppg_grn'], MotherWavelet_ppg, level=LevelFour, axis=0)
        ppgApprCoef = PPGCoef[LevelFour]
      
        # Frequency Domain PPG 
        maxFPPG.append(np.max(ppgApprCoef))
        minFPPG.append(np.min(ppgApprCoef))
        meanFPPG.append(np.mean(ppgApprCoef))
        stdFPPG.append(np.std(ppgApprCoef))
        maxminFPPG.append(np.ptp(ppgApprCoef))
        kurFPPG.append(ss.kurtosis(ppgApprCoef))
        skewFPPG.append(ss.skew(ppgApprCoef))
        rmsFPPG.append(rms(ppgApprCoef))
        crestFPPG.append(crestF(ppgApprCoef))
        shapeFPPG.append(shapeF(ppgApprCoef))
        impulseFPPG.append(impulseF(ppgApprCoef))
        
        # Time Domain GSR
        maxGSR.append(np.max(data1['GSR']))
        minGSR.append(np.min(data1['GSR']))
        meanGSR.append(np.mean(data1['GSR']))
        stdGSR.append(np.std(data1['GSR']))
        maxminGSR.append(np.ptp(data1['GSR']))
        kurGSR.append(ss.kurtosis(data1['GSR']))
        skewGSR.append(ss.skew(data1['GSR']))
        rmsGSR.append(rms(data1['GSR']))
        crestGSR.append(crestF(data1['GSR']))
        shapeGSR.append(shapeF(data1['GSR']))
        impulseGSR.append(impulseF(data1['GSR']))
        
        
        # Frequency Domain GSR
        LevelTen = 10
        GSRCoefTen = pywt.wavedec(data1['GSR'], MotherWavelet_gsr, level=LevelTen, axis = 0)
        zero_out_detailed(GSRCoefTen, 10)
        recon_ten = pywt.waverec(GSRCoefTen, 'db3')

        LevelEight = 8 
        GSRCoefEight = pywt.wavedec(data1['GSR'], MotherWavelet_gsr, level=LevelEight, axis = 0)
        zero_out_detailed(GSRCoefTen, 8)
        recon_eight = pywt.waverec(GSRCoefEight, 'db3')

        LevelSix = 6
        GSRCoefSix = pywt.wavedec(data1['GSR'], MotherWavelet_gsr, level=LevelSix, axis = 0)
        zero_out_detailed(GSRCoefSix, 6)
        recon_six = pywt.waverec(GSRCoefSix, 'db3') 
        
        amp_FGSR, val_FGSR = get_phasic_amp_and_max(data1['GSR'].values, recon_ten)
        zeroCrossScvsrFGSR.append(get_scvsr_zcross(recon_eight, recon_ten))       # SCVSR = A8 - A10
        zeroCrossScsrFGSR.append(get_scsr_zcross(recon_eight, recon_ten))         # SCSR = A6 - A10
        phasicMaxAmpFGSR.append(amp_FGSR)                                         # Phasic = GSR - A10 
        phasicMaxValFGSR.append(val_FGSR)                                         # Phasic = GSR - A10 
        
        # Time Domain Skin Temperatrue
        maxSKT.append(np.max(data1['SKT']))
        minSKT.append(np.min(data1['SKT']))
        meanSKT.append(np.mean(data1['SKT']))
        stdSKT.append(np.std(data1['SKT']))
        maxminSKT.append(np.ptp(data1['SKT']))
        kurSKT.append(ss.kurtosis(data1['SKT']))
        skewSKT.append(ss.skew(data1['SKT']))
        rmsSKT.append(rms(data1['SKT']))
        crestSKT.append(crestF(data1['SKT']))
        shapeSKT.append(shapeF(data1['SKT']))
        impulseSKT.append(impulseF(data1['SKT']))
        
        # Emotion Label
        valence.append(np.mean(data1['valence']))
        arousal.append(np.mean(data1['arousal']))
        label_valence.append(int(np.mean(data1['valence'])*3/10.01)-1)
        label_arousal.append(int(np.mean(data1['arousal'])*3/10.01)-1)

    result = {'meanNN': meanNN,
              'medianNN': medianNN,
              'SDNN': SDNN,
              'pNN50': pNN50,
              'NN50': NN50,
              'RMSSD': RMSSD,  # ,'HR':HR

              'maxFPPG': maxFPPG,
              'minFPPG': minFPPG,
              'meanFPPG': meanFPPG,
              'stdFPPG': stdFPPG,
              'maxminFPPG': maxminFPPG,
              'kurFPPG': kurFPPG,
              'skewFPPG': skewFPPG,
              'rmsFPPG': rmsFPPG,
              'crestFPPG': crestFPPG,
              'shapeFPPG': shapeFPPG,
              'impulseFPPG': impulseFPPG,

              'maxGSR': maxGSR,
              'minGSR': minGSR,
              'meanGSR': meanGSR,
              'stdGSR': stdGSR,
              'maxminGSR': maxminGSR,
              'kurGSR': kurGSR,
              'skewGSR': skewGSR,
              'rmsGSR': rmsGSR,
              'crestGSR': crestGSR,
              'shapeGSR': shapeGSR,
              'impulseGSR': impulseGSR,
              
              'zeroCrossScvsrFGSR': zeroCrossScvsrFGSR,
              'zeroCrossScsrFGSR': zeroCrossScsrFGSR,
              'phasicAmpFGSR': phasicMaxAmpFGSR,
              'phasicMaxFGSR': phasicMaxValFGSR, 

              'maxSKT': maxSKT,
              'minSKT': minSKT,
              'meanSKT': meanSKT,
              'stdSKT': stdSKT,
              'maxminSKT': maxminSKT,
              'kurSKT': kurSKT,
              'skewSKT': skewSKT,
              'rmsSKT': rmsSKT,
              'crestSKT': crestSKT,
              'shapeSKT': shapeSKT,
              'impulseSKT': impulseSKT,

              'valence': valence,
              'arousal': arousal,
              'label_valence': label_valence,
              'label_arousal': label_arousal

              }

    return result
    


In [6]:
"""
working_data, measures = hp.process(data['ppg_grn'].values, fs, calc_freq=True)
result = pd.DataFrame(NN_dictionary(working_data, time_interval, data))
print(result)
"""
#result.to_csv('timefeaturetest.csv', index = False)
#sns.pairplot(result)
#plt.show()

fs = 398
time_interval = 11940
strd = 1

Dir = 'C:/Users/Jihwan/Desktop/Test_Data' #경로 설정
outputDir = 'C:/Users/Jihwan/Desktop/Processed/'
os.chdir(Dir)
filelist = os.listdir(Dir) #Dir 경로 폴더 안 모든 파일 이름 리스트

for file in filelist:
    data = pd.read_csv(file, header = None)
    data.columns = colname
    result = pd.DataFrame(features2dictionary(fs, time_interval, data, strd))
    result.to_csv(outputDir + file, index = False)


row start:: 0 row end:: 11940
shape:: (11940, 5)
row start:: 398 row end:: 12338
shape:: (11940, 5)
row start:: 796 row end:: 12736
shape:: (11940, 5)
row start:: 1194 row end:: 13134
shape:: (11940, 5)
row start:: 1592 row end:: 13532
shape:: (11940, 5)
row start:: 1990 row end:: 13930
shape:: (11940, 5)
row start:: 2388 row end:: 14328
shape:: (11940, 5)
row start:: 2786 row end:: 14726
shape:: (11940, 5)
row start:: 3184 row end:: 15124
shape:: (11940, 5)
row start:: 3582 row end:: 15522
shape:: (11940, 5)
row start:: 3980 row end:: 15920
shape:: (11940, 5)
row start:: 4378 row end:: 16318
shape:: (11940, 5)
row start:: 4776 row end:: 16716
shape:: (11940, 5)
row start:: 5174 row end:: 17114
shape:: (11940, 5)
row start:: 5572 row end:: 17512
shape:: (11940, 5)
row start:: 5970 row end:: 17910
shape:: (11940, 5)
row start:: 6368 row end:: 18308
shape:: (11940, 5)
row start:: 6766 row end:: 18706
shape:: (11940, 5)
row start:: 7164 row end:: 19104
shape:: (11940, 5)
row start:: 7562 

C:\Users\Jihwan\anaconda3\lib\site-packages\scipy\interpolate\fitpack2.py:279: UserWarning: 
The maximal number of iterations maxit (set to 20 by the program)
allowed for finding a smoothing spline with fp=s has been reached: s
too small.
There is an approximation returned but the corresponding weighted sum
of squared residuals does not satisfy the condition abs(fp-s)/s < tol.
  warnings.warn(message)


row start:: 37014 row end:: 48954
shape:: (11940, 5)
row start:: 37412 row end:: 49352
shape:: (11940, 5)
row start:: 37810 row end:: 49750
shape:: (11940, 5)
row start:: 38208 row end:: 50148
shape:: (11940, 5)
row start:: 38606 row end:: 50546
shape:: (11940, 5)
row start:: 39004 row end:: 50944
shape:: (11940, 5)
row start:: 39402 row end:: 51342
shape:: (11940, 5)
row start:: 39800 row end:: 51740
shape:: (11940, 5)
row start:: 40198 row end:: 52138
shape:: (11940, 5)
row start:: 40596 row end:: 52536
shape:: (11940, 5)
row start:: 40994 row end:: 52934
shape:: (11940, 5)
row start:: 41392 row end:: 53332
shape:: (11940, 5)
row start:: 41790 row end:: 53730
shape:: (11940, 5)
row start:: 42188 row end:: 54128
shape:: (11940, 5)
row start:: 42586 row end:: 54526
shape:: (11940, 5)
row start:: 42984 row end:: 54924
shape:: (11940, 5)
row start:: 43382 row end:: 55322
shape:: (11940, 5)
row start:: 43780 row end:: 55720
shape:: (11940, 5)
row start:: 44178 row end:: 56118
shape:: (119

C:\Users\Jihwan\anaconda3\lib\site-packages\scipy\interpolate\fitpack2.py:279: UserWarning: 
The maximal number of iterations maxit (set to 20 by the program)
allowed for finding a smoothing spline with fp=s has been reached: s
too small.
There is an approximation returned but the corresponding weighted sum
of squared residuals does not satisfy the condition abs(fp-s)/s < tol.
  warnings.warn(message)


row start:: 54526 row end:: 66466
shape:: (11542, 5)
row start:: 54924 row end:: 66864
shape:: (11144, 5)
row start:: 55322 row end:: 67262
shape:: (10746, 5)
row start:: 55720 row end:: 67660
shape:: (10348, 5)
row start:: 56118 row end:: 68058
shape:: (9950, 5)
row start:: 56516 row end:: 68456
shape:: (9552, 5)
row start:: 56914 row end:: 68854
shape:: (9154, 5)
row start:: 57312 row end:: 69252
shape:: (8756, 5)
row start:: 57710 row end:: 69650
shape:: (8358, 5)
row start:: 58108 row end:: 70048
shape:: (7960, 5)
row start:: 58506 row end:: 70446
shape:: (7562, 5)
row start:: 58904 row end:: 70844
shape:: (7164, 5)
row start:: 59302 row end:: 71242
shape:: (6766, 5)
row start:: 59700 row end:: 71640
shape:: (6368, 5)
row start:: 60098 row end:: 72038
shape:: (5970, 5)
row start:: 60496 row end:: 72436
shape:: (5572, 5)
row start:: 60894 row end:: 72834
shape:: (5174, 5)
row start:: 61292 row end:: 73232
shape:: (4776, 5)
row start:: 61690 row end:: 73630
shape:: (4378, 5)
row star

C:\Users\Jihwan\anaconda3\lib\site-packages\pywt\_multilevel.py:43: UserWarning: Level value of 10 is too high: all coefficients will experience boundary effects.
  warnings.warn(
C:\Users\Jihwan\anaconda3\lib\site-packages\pywt\_multilevel.py:43: UserWarning: Level value of 10 is too high: all coefficients will experience boundary effects.
  warnings.warn(
C:\Users\Jihwan\anaconda3\lib\site-packages\pywt\_multilevel.py:43: UserWarning: Level value of 10 is too high: all coefficients will experience boundary effects.
  warnings.warn(
C:\Users\Jihwan\anaconda3\lib\site-packages\pywt\_multilevel.py:43: UserWarning: Level value of 10 is too high: all coefficients will experience boundary effects.
  warnings.warn(
C:\Users\Jihwan\anaconda3\lib\site-packages\pywt\_multilevel.py:43: UserWarning: Level value of 10 is too high: all coefficients will experience boundary effects.
  warnings.warn(
C:\Users\Jihwan\anaconda3\lib\site-packages\pywt\_multilevel.py:43: UserWarning: Level value of 10 i

row start:: 62486 row end:: 74426
shape:: (3582, 5)
row start:: 62884 row end:: 74824
shape:: (3184, 5)
row start:: 63282 row end:: 75222
shape:: (2786, 5)
row start:: 63680 row end:: 75620
shape:: (2388, 5)
row start:: 64078 row end:: 76018
shape:: (1990, 5)
row start:: 64476 row end:: 76416
shape:: (1592, 5)
row start:: 64874 row end:: 76814
shape:: (1194, 5)
row start:: 65272 row end:: 77212
shape:: (796, 5)
row start:: 65670 row end:: 77610
shape:: (398, 5)
row start:: 66068 row end:: 78008
shape:: (0, 5)


C:\Users\Jihwan\anaconda3\lib\site-packages\pywt\_multilevel.py:43: UserWarning: Level value of 8 is too high: all coefficients will experience boundary effects.
  warnings.warn(
C:\Users\Jihwan\anaconda3\lib\site-packages\pywt\_multilevel.py:43: UserWarning: Level value of 10 is too high: all coefficients will experience boundary effects.
  warnings.warn(
C:\Users\Jihwan\anaconda3\lib\site-packages\pywt\_multilevel.py:43: UserWarning: Level value of 8 is too high: all coefficients will experience boundary effects.
  warnings.warn(
